#  GPU_CAGRA
- GPU_CAGRA 인덱스는 GPU에 최적화된 그래프 기반 인덱스
- 추론용 GPU를 사용하여 Milvus GPU 버전을 실행하는 것이 값비싼 훈련용 GPU를 사용하는 것보다 비용 효율적일 수 있음

참고자료: https://milvus.io/docs/ko/gpu-cagra.md#Index-building-params

## 인덱스 구축
- Milvus의 벡터 필드에 GPU_CAGRA 인덱스를 구축하려면 add_index() 방법을 사용하여 index_type, metric_type 및 인덱스에 대한 추가 매개변수를 지정

In [21]:
from pymilvus import MilvusClient, DataType

index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="vector",
    index_type="GPU_CAGRA",
    index_name="vector_index",
    metric_type="L2",
    params={
        "intermediate_graph_degree": 32, # 가지치기 전 그래프의 정도를 결정하여 리콜 및 빌드 시간에 영향을 줌 32, 64 사용
        "graph_degree": 64, # 가지치기 후 그래프의 차수를 설정하여 성능과 리콜에 영향을 줌 intermediate_graph_degree보다 작아야함
        "build_algo": "IVF_PQ", # 가지치기 전 그래프 생성 알고리즘을 선택, IVF_PQ: 더 높은 품질을 제공하지만 빌드 시간이 느림, NN_DESCENT: 더 빠른 빌드를 제공하지만 더 낮은 리콜을 제공
        "cache_dataset_on_device": "true", # 원본 데이터세트를 GPU 메모리에 캐시할지 여부 결정
        "adapt_for_cpu": "false", # 인덱스 구축에 GPU를 사용할지, 검색에 CPU를 사용할지 결정 이 파라미터를 true로 설정하려면 검색 요청에 ef 파라미터가 있어야함
    }
)

## 인덱스에서 검색
- 인덱스가 구축되고 엔티티가 삽입되면 인덱스에서 유사도 검색을 수행할 수 있음

In [22]:
search_params = {
    "params": {
        "itopk_size": 16, # Determines the size of intermediate results kept during the search
        "search_width": 8, # Specifies the number of entry points into the CAGRA graph during the search
    }
}

client = MilvusClient(
    uri="http://localhost:19530"
)

if client.has_collection("my_collection"):
    client.drop_collection("my_collection")

schema = client.create_schema(
    auto_id=False,
    enable_dynamic_schema=False,
)

schema.add_field(field_name="index", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=5)
schema.add_field(field_name="scalar_1", datatype=DataType.INT64)
schema.add_field(field_name="scalar_2", datatype=DataType.INT32)

client.create_collection(
    collection_name="my_collection",
    schema=schema,
    index_params=index_params
)

client.load_collection("my_collection")

res = client.search(
    collection_name="my_collection", # Collection name
    anns_field="vector", # Vector field name
    data=[[0.1, 0.2, 0.3, 0.4, 0.5]],  # Query vector
    limit=3,  # TopK results to return
    search_params=search_params
)

2025-07-08 04:12:30,227 [ERROR][handler]: RPC error: [create_index], <MilvusException: (code=1100, message=invalid parameter[expected=valid index][actual=invalid index type: GPU_CAGRA])>, <Time:{'RPC start': '2025-07-08 04:12:30.225007', 'RPC error': '2025-07-08 04:12:30.227241'}> (decorators.py:140)
2025-07-08 04:12:30,227 [ERROR][_create_index]: Failed to create an index on collection: my_collection (milvus_client.py:183)


MilvusException: <MilvusException: (code=1100, message=invalid parameter[expected=valid index][actual=invalid index type: GPU_CAGRA])>